# Nuoseklių sakinių klasifikavimas atsitiktinių imčių medicininių tyrimų santraukose

## Santrauka

**Tikslas:** sukurti natūralios kalbos apdorojimo (*angl. Natural Language Processing*, **NLP**) modelį, kuris palengvintų medicininių tyrimų publikacijų santraukų skaitymą.

**Metodai:**
- Google TensorFlow modelių kūrimas

**Rezultatas:**

**Išvada:**

### Bibliotekų importavimas

In [6]:
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

2023-03-13 18:23:55.285096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Įvadas

Kiekvienais metais publikuojamų medicininių tyrimų skaičius tik auga, todėl medicinos specialistams tampa vis sunkiau greitai aptikti jiems reikiamą informaciją. Tyrinėtojai nagrinėdami ankstesnes publikacijas, dažniausiai perskaito tik santraukas, norėdami greitai patikrinti, ar straipsniai atitinka jų kriterijus. Šis procesas yra lengvesnis ir greitesnis jei santrauka yra **struktūrizuota**, t.y. padalinima į semantines antraštes (etikėtes), tokias kaip **tikslas, metodas, rezultatas ir išvada**. 

Kiekvieno santraukos sakinio suskirstymas į atitinkamą antraštę vadinamas nuoseklių sakinių klasifikavimu. Šiai  užduočiai atlikti šiame darbe įgyvendinama dirbtinio neuroninio tinklo (*angl. artificial neural network*, **ANN**) modeliu pagrįsta sistema.

Darbas atliekamas remiantis **Franck Dernoncourt**, **Ji Young Lee** ir **Peter Szolovits** 2017 metų publikacija "[**Neural Networks for Joint Sentence Classification in Medical Paper Abstracts**](https://arxiv.org/pdf/1612.05251)".

Neuroninių modelių kūrimui naudojama atviro kodo mašininio mokymo ir dirbtinio intelekto biblioteka [**Google TensorFlow**](https://www.tensorflow.org/).

## Duomenų rinkinys (*angl. dataset*)


[**PubMed 200k RCT**](https://arxiv.org/pdf/1710.06071.pdf) duomenų rinkinys sudaryta iš viešai prieinamos bibliotekos [PubMed](https://pubmed.ncbi.nlm.nih.gov/) įrašų ir yra skirtas nuosekliam sakinių klasifikavimui atsitiktinės imties kontroliuojamų medicininių tyrimų santraukose. Duomenų rinkinys susideda iš ~200 tūkst. santraukų, kurias sudaro apie  2,3 mln. sakinių.

Kiekvienas sakinys yra pažymėtas etikėte, nurodančia sakinio reikšmę santraukoje:
- **background** - kilmė, ištakos
- **objective** - tikslas
- **methods** - metodai
- **results** - rezultatai
- **conclusions** - išvados

Rinkinys yra viešai prieinamas [**GitHub repozitorijoje**](https://github.com/Franck-Dernoncourt/pubmed-rct)

### Atsiuntimas

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct

### Struktūra

In [5]:
!ls pubmed-rct

PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


Pagrindinis duomenų rinkinys **PubMed 200k RCT** kai kuriais atvejais gali būti per didelis, todėl papildomai yra pateikiamas **PubMed 20k RCT** rinkinys, sudarytas iš 10% pagrindiniame rinkinyje esančių vėliausiai publikuotų tyrimų santraukų.

Šalia duomenų rinkinių su originaliu tekstu, pateikiamos versijos, kuriose santraukų tekste esantys skaičiai pakeisti @ simboliu.

In [3]:
!ls pubmed-rct/PubMed_20k_RCT/

dev.txt  test.txt  train.txt


In [4]:
!ls pubmed-rct/PubMed_200k_RCT/

dev.txt  test.txt  train.7z


Rinkinio aplankuose yra po tris failus:
- **train.txt** (**train.7z**) - mokymo santraukų rinkinys,
- **test.txt** - testavimo santraukų rinkinys,
- **dev.txt** - modelio įvertinimo (*angl. validation*) rinkinys.

### Failo formatas

In [4]:
def get_all_lines(filename: str) -> list:
    """
    Grąžina nuskaityto tekstinio failo  filename  eilutes string sąraše
    """

    with open(filename, "r") as f:
        return f.readlines()

Kiekvienas failas turi tą patį formatą:
- **PMID eilutė** - santraukos pradžia, publikacijos *identifikavimo numeris PubMed* bibliotekoje;
- **santraukos sakiniai** atskirose eilutėse, kiekvienos pradžioje didžiosiomis raidėmis sakinio etiketė ir santraukos sakinio tekstas atskirti tabuliacijos simboliu (*angl. tab*);
- **tuščia eilutė** - santraukos pabaiga.


In [7]:
FOLDER_PATH = "pubmed-rct/PubMed_20k_RCT/"

filename = os.path.join(FOLDER_PATH, "test.txt")
file_lines = get_all_lines(filename)

# išvedam 20 pirmų failo eilučių
file_lines[:20]

['###24845963\n',
 'BACKGROUND\tThis study analyzed liver function abnormalities in heart failure patients admitted with severe acute decompensated heart failure ( ADHF ) .\n',
 'RESULTS\tA post hoc analysis was conducted with the use of data from the Evaluation Study of Congestive Heart Failure and Pulmonary Artery Catheterization Effectiveness ( ESCAPE ) .\n',
 'RESULTS\tLiver function tests ( LFTs ) were measured at 7 time points from baseline , at discharge , and up to 6 months follow-up .\n',
 'RESULTS\tSurvival analyses were used to assess the association between admission Model of End-Stage Liver Disease Excluding International Normalized Ratio ( MELD-XI ) scores and patient outcome.There was a high prevalence of abnormal baseline ( admission ) LFTs ( albumin 23.8 % , aspartate transaminase 23.5 % , alanine transaminase 23.8 % , and total bilirubin 36.1 % ) .\n',
 "RESULTS\tThe percentage of patients with abnormal LFTs decreased significantly from baseline to 6-months ' follow-u

### Duomenų formato keitimas

Patogesniam darbui su duomenų rinkiniu, perkelsime duomenys į **pandas DataFrame**.

Tarpiniam formatui naudosim python žodyną:
- **number** - sakinio numeris santraukoje
- **target** - sakinio etiketė
- **text**   - sakinio tekstas
- **total**  - visų santraukoje esančių sakinių skaičius

```
[{'target': 'BACKGROUND',
  'text': 'This study analyzed liver function abnormalities in heart failure patients admitted with severe acute decompensated heart failure ( ADHF ) .\n',
  'number': 0,
  'total': 9},
 {...},
]
```

In [45]:
def convert_file_lines_to_dic(filename: str) -> list:
    """
    Grąžina faile  filename  esančių santraukų sakinių žodynų sąrašą.
    Sakinio žodynas papildomas eilės numeriu ir visų santraukos sakinių skaičiumi.
    """
        
    all_lines = get_all_lines(filename) #nuskaitom visas faile esančias eilutes
    list_of_dicts = []

    for line in all_lines:
        if line.startswith("###"):  #tikrinam ar eilutė prasideda simboliais ### (santraukos pradžia)
            abstract = ''           
        
        elif line.isspace():        #tikrinam ar eilutė yra tuščia (santraukos pabaiga)
            abstract_lines = abstract.splitlines()
            total = len(abstract_lines)

            
            for i, sentence in enumerate(abstract_lines):
                sentence_parts = sentence.split('\t')   #[0] etiketė, [1] tekstas

                sentence_dict = {}

                sentence_dict['target'] = sentence_parts[0]
                sentence_dict['text'] = sentence_parts[1]
                sentence_dict['number'] = i
                sentence_dict['total'] = total
                
                list_of_dicts.append(sentence_dict)
       
        else:                       #pridedam eilutę prie kitų santraukos eilučių
            abstract += line
    
    return list_of_dicts

In [46]:
# Perkeliam duomenų rinkinio failų santraukų sakinius į žodynų sąrašus
DATASET_PATH = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

train_dicts = convert_file_lines_to_dic(DATASET_PATH + 'train.txt')
test_dicts = convert_file_lines_to_dic(DATASET_PATH + 'test.txt')
val_dicts = convert_file_lines_to_dic(DATASET_PATH + 'dev.txt')

In [47]:
# Patikriname pirmus penkis train.txt faile esančius sakinius
train_dicts[:5]

[{'target': 'OBJECTIVE',
  'text': 'To investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .',
  'number': 0,
  'total': 12},
 {'target': 'METHODS',
  'text': 'A total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .',
  'number': 1,
  'total': 12},
 {'target': 'METHODS',
  'text': 'Outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
  'number': 2,
  'total': 12},
 {'target': 'METHODS',
  'text': 'Pain was assessed using the visual analog pain scale ( @-@ mm ) .',
  'number': 3,
  'total': 12},
 {'target': 'METHODS',
  'text': 'Secondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis I

Nauju formatu turimus duomenys galime lengvai perkelti į pandas DataFrame.

In [48]:
train_df = pd.DataFrame(train_dicts)
test_df = pd.DataFrame(test_dicts)
val_df = pd.DataFrame(val_dicts)

In [49]:
train_df.head()

,target,text,number,total
0,OBJECTIVE,To investigate the efficacy of @ weeks of dail...,0,12
1,METHODS,A total of @ patients with primary knee OA wer...,1,12
2,METHODS,Outcome measures included pain reduction and i...,2,12
3,METHODS,Pain was assessed using the visual analog pain...,3,12
4,METHODS,Secondary outcome measures included the Wester...,4,12


### Duomenų suskirstymas rinkinio failuose

**PubMed 200k RCT** sudaro 195 654 santraukų:
- **train** - 190654
- **test** - 2500
- **validation** - 2500

**PubMed 20k RCT** sudaro 20 tūkst. santraukų:
- **train** - 15000
- **test** - 2500
- **validation** - 2500


## Bazinis modelis